# Reinforcement learning in Markov decision processes

What we learn :
- learn how to solve an MDP without knowing the transition model and the rewards but we just have the ability to interact with the environment and receive transitions.
- Q-learning : converge, family of algorithms
- Exploration-Exploitation : learn and use

Let's first introduce things with a reinforcement learning (RL) API (Application Programmer Interface). 
- PLANNING : Previously we saw the planner with the idea of being able to take a model of an MDP (Markov Decision Process) which consists of a transition function T and a reward function R and it goes through some code (planner) and a policy comes out. A policy $\pi$, figures how to behave to maximize reward and it maps states to actions.
- RL : Instead of taking a model as input a learner is going to take samples of being in some states, taking some actions, observing a reward amd observing the state that is at the other end of that transition. Then instead of computing a policy we're going to learn a policy.
![](api.png)

## Approches to RL

- The first box $\pi$ maps states to actions and this is called a policy. Algorithms that work directly on trying to find the policy are called policy search algorithms. The learning here is indirect.
- The second box $U$ maps states to values and is called utility and algorithms that target that are called value-function based algorithms. The learning here is not quite as indirect than with the policy search.
- The third takes states and the transition model gives the next states and it takes actions and the reward function returns rewards. This is called model based learner.

![](rl-approches.png)

Now have a new kind of value function. Before we saw :

U is defined as the long-term value of being in a state and it is equal to the reward $R(s)$ for arriving in that state plus the discounted reward of the futur. We're going to choose some action, then we're going to take an expectation over all the possible next states and we're going to arrive in some next state s'. This is recursive and non linear. but we can use value iteration to do that.

$$U(s) = R(s) + \gamma \; max_a \sum_{s'} \; T(s, a, s')\;U(s')$$

The policy in a state s, we'll consider all the actions we can take to leave that state, we'll look to what their expected values are. So we'll iterate over all the possible next states weighted by their probability of the utlity of landing in the state that we'd end up in.

$$\pi(s) = argmax_a \; \sum_{s'}\; T(s, a, s')\;U(s')$$

Here is a new kind of function called the Q function. This is the value for arriving in some state S and R(s) is the reward for we get for that arrival. Then we're going to leave s via action a. So we're going to add the discounted expected value that we get for taking action a and once we land in s', we're going to take whichever action has the highest Q value from there. This can be resumed as : "value for arriving in s, leaving via a, proceeding optimally thereafter".

It turns out that into the Q function we have everything we need for dealing with $U$ and $\pi$ without knowing the transition function T or the reward function R.

$$Q(s,a) = R(s) + \gamma \; \sum_{s'} \; T(s, a, s') \; max_{a'}\; Q(s', a')$$

To prove this were going to answer a little quiz: Use Q to define U and $\pi$

U(s) returns a value (scalar) and $\pi(s)$ returns an action. We could turn Q into U if we always pick the best action. The best action is the one that maximizes the value that you're going to get from that point on. So 
$$U(s) = max_a Q(s, a)$$

The same way, the policy that you want to follow is the one that maximizes your value going forward except the differences. It's returning an A and not an actual value so it should be argmax.
$$\pi(s) = argmax_a Q(s, a)$$

Finding Q is called Q learning


## Estimating Q from transitions

Here is the Q equation : 
$$Q(s,a) = R(s) + \gamma \; \sum_{s'} \; T(s, a, s') \; max_{a'}\; Q(s', a')$$

And we can't solve this because we don't have access to R and T. All we have access to are transitions. So here is how we are going to use transitions. This is a transition \<s, a, r, s'\>, we were in some state s of the MDP then action a was chosen, then the transition happens and we land in a state, we get the reward for landing in that state and we find out what state we're in. What we are going to do with it ? We're going to imagine that we've got an estimate of the $Q$ function, $\hat Q$, and we're going to update it as follows:

$$\hat Q(s, a) \xleftarrow{\alpha_t} r + \gamma \; max_{a'} \; \hat Q(s', a')$$

We're going to take the state and action that we just experienced and we're going to update it, we're going to move it a little bit ($\alpha =$ learning rate) in the direction of the immediate reward $r$ plus the discounted $\gamma$ estimated value of the next state. So we're going to take our estimate $\hat Q$, we're going to take the state that we end up in $s'$ and we're going to look at all the different actions we could take from there and take the maximum. So this together is kind of an estimate of the utility $max_{a'} \; \hat Q(s', a')$. And this is the utility of the state we are going to. This altogether is the utility of the state that we're in. Note that $V \xleftarrow{\alpha_t} X \equiv V \leftarrow (1 - \alpha)V + \alpha X $. When $\alpha = 0 $ the learning rate is 0 and this corresponds to no learning because we assign V to V and when $\alpha = 1 $, it's like full learning so we forget everything that we knew and jump to a new value and assign X to V.

The equation above computes the expected value for following the optimal policy after you take this particular action
$$\mathbb E$$



## Q-learning convergence

If we start $\hat Q$ off pretty much anywhere, and then update it according to the rule discussed, then we update the Q value for (s, a) move it $\alpha$ of the way towards r plus $\gamma$ max a of the Q value of s'. As long as we do that, the estimate $\hat Q (s, a)$ goes to $Q(s, a)$ the actual solution to the Bellman equation. This is only true if we actually visit s, a infinitely often. The learning rates have to be updated the way discussed before. The next state needs to be drawn form the actual transition probabilities and the rewards need to be drawn from the reward function. 

$\hat Q$ starts anywhere $\hat Q(s, a) \xleftarrow{\alpha_t} r + \gamma \; max_{a'} \; \hat Q(s', a')$ then $\hat Q(s, a) \xleftarrow{\alpha_t} Q(s, a)$ if s, a visited infinitely often $\sum_t \alpha_t = \infty, \sum_t \alpha_t^2 < \infty$$s' \sim T(s, a, s'), r \sim R(s)$

## Choosing actions

Q-learning is a family of algorithms:
- how initialize $\hat Q$ ?
- how decay $\alpha_t$
- how choose actions ?

Different ways of making these choices will lead to algorithms with fairly different behavior. For the example of how choose actions, we can't always choose randomly because then we would learn the best policy but never actually use it , meanwhile always choosing the same actions will make us not learn anything. Using $\hat Q$ everytime is the best option but has the downside of possibly falling into a local minima. To avoid this, we can do **simulated annealing** approches that take a random action once in a while.
$$\hat \pi (s) = argmax_a \; \hat Q(s, a) \qquad with \; probability \; 1 - \epsilon $$

## $\epsilon$-greedy exploration

If the action selection is GLIE (Greedy in the Limit with Infinite Exploration) and that means we're decaying our $\epsilon$. We start off more random and over time we get less and less random and more and more greedy. Then we have two things that are true, one is that $\hat Q$ goes to Q and the policy that we're following $\hat \pi$ is getting more and more like the optimal policy $\pi^*$ over time. So not only do we learn stuff but we use it too. This is an example of the exploration-exploitation dilemma.